## Import packages

In [1]:
import numpy as np
import json
import pickle
import nltk
from tqdm import tqdm, tqdm_notebook

In [2]:
DATA_PATH = 'dataset/'

## Preprocess training data

### Load SQuAD 1.0 training & developing set

In [3]:
file_name = DATA_PATH + "train-v1.1.json"
with open(file_name) as json_data:
    training_set = json.load(json_data)
    training_set = training_set['data']

In [4]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(training_set[0])

{   'paragraphs': [   {   'context': 'Architecturally, the school has a '
                                     'Catholic character. Atop the Main '
                                     "Building's gold dome is a golden statue "
                                     'of the Virgin Mary. Immediately in front '
                                     'of the Main Building and facing it, is a '
                                     'copper statue of Christ with arms '
                                     'upraised with the legend "Venite Ad Me '
                                     'Omnes". Next to the Main Building is the '
                                     'Basilica of the Sacred Heart. '
                                     'Immediately behind the basilica is the '
                                     'Grotto, a Marian place of prayer and '
                                     'reflection. It is a replica of the '
                                     'grotto at Lourdes, France where the '

                                                            'text': '3,577'}],
                                         'id': '5733ae924776f41900661014',
                                         'question': 'How many incoming '
                                                     'students did Notre Dame '
                                                     'admit in fall 2015?'},
                                     {   'answers': [   {   'answer_start': 138,
                                                            'text': '19.7%'}],
                                         'id': '5733ae924776f41900661013',
                                         'question': 'What percentage of '
                                                     'students were admitted '
                                                     'to Notre Dame in fall '
                                                     '2015?'},
                                     {   'answers': [   {   'answer_start': 213,
    

                          'qas': [   {   'answers': [   {   'answer_start': 71,
                                                            'text': '8,448'}],
                                         'id': '5733b5df4776f41900661105',
                                         'question': 'How many undergrads were '
                                                     'attending Notre Dame in '
                                                     '2014?'},
                                     {   'answers': [   {   'answer_start': 196,
                                                            'text': '21–24%'}],
                                         'id': '5733b5df4776f41900661106',
                                         'question': 'What percentage of '
                                                     'students at Notre Dame '
                                                     'are the children of '
                                                     'former Notre Dame '
  

                                         'id': '573387acd058e614000b5cb4',
                                         'question': 'How many national titles '
                                                     'were won when Knute '
                                                     'Rockne coached at Notre '
                                                     'Dame?'}]},
                      {   'context': 'The success of its football team made '
                                     'Notre Dame a household name. The success '
                                     'of Note Dame reflected rising status of '
                                     'Irish Americans and Catholics in the '
                                     '1920s. Catholics rallied up around the '
                                     'team and listen to the games on the '
                                     'radio, especially when it knocked off '
                                     'the schools that symbolized the '
  

                          'qas': [   {   'answers': [   {   'answer_start': 56,
                                                            'text': 'Fr. '
                                                                    'Zahm'}],
                                         'id': '573398ebd058e614000b5e66',
                                         'question': 'Which person oversaw the '
                                                     'creation of a science '
                                                     'hall at Notre Dame in '
                                                     '1883?'},
                                     {   'answers': [   {   'answer_start': 73,
                                                            'text': '1950'}],
                                         'id': '573398ebd058e614000b5e67',
                                         'question': 'In what year did the '
                                                     'student union building '
   

                                         'id': '573382a14776f41900660c2d',
                                         'question': 'The school known as '
                                                     'Notre Dame is known by a '
                                                     'more lengthy name, what '
                                                     'is it?'},
                                     {   'answers': [   {   'answer_start': 92,
                                                            'text': 'Catholic '
                                                                    'research '
                                                                    'university'}],
                                         'id': '573382a14776f41900660c2e',
                                         'question': 'What type of institution '
                                                     'is the Notre Dame?'},
                                     {   'answers': [   {   'answer_

                                     'all of its sports except football and '
                                     'fencing changed in July 2013 as a result '
                                     'of major conference realignment, and its '
                                     'fencing affiliation will change in July '
                                     '2014. The Irish left the Big East for '
                                     'the ACC during a prolonged period of '
                                     'instability in the Big East; while they '
                                     'maintain their football independence, '
                                     'they have committed to play five games '
                                     'per season against ACC opponents. In ice '
                                     'hockey, the Irish were forced to find a '
                                     'new conference home after the Big Ten '
                                     "Conference'

                                     'wins against the top team. The team '
                                     'plays in newly renovated Purcell '
                                     'Pavilion (within the Edmund P. Joyce '
                                     'Center), which reopened for the '
                                     'beginning of the 2009–2010 season. The '
                                     'team is coached by Mike Brey, who, as of '
                                     'the 2014–15 season, his fifteenth at '
                                     'Notre Dame, has achieved a 332-165 '
                                     'record. In 2009 they were invited to the '
                                     'NIT, where they advanced to the '
                                     'semifinals but were beaten by Penn State '
                                     'who went on and beat Baylor in the '
                                     'championship. The 2010–11 team concluded '


In [9]:
def get_clean_dataset(dataset):
    
    contexts = dict()
    questions = []
    
    context_counter = 0
    question_counter = 0
    
    for article in tqdm_notebook(dataset):
        for para in article["paragraphs"]:
            
            contexts[context_counter] = para["context"]
            for q in para["qas"]:
                questions.append({
                                                  "question_id": question_counter,
                                                  "context_id": context_counter,
                                                  "question": q["question"],
                                                  "answers": [{'text': ans["text"], 
                                                               'answer_start': para["context"][:ans["answer_start"]]} 
                                                                    for ans in q["answers"]]
                                })
                question_counter += 1

            context_counter += 1

    return contexts, questions

In [10]:
train_c, train_q = get_clean_dataset(training_set)

In [11]:
train_q[0]

{'question_id': 0,
 'context_id': 0,
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': [{'text': 'Saint Bernadette Soubirous',
   'answer_start': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to '}]}

In [12]:
with open(DATA_PATH + 'trainset_context.pickle', 'wb') as handle:
    pickle.dump(train_c, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(DATA_PATH + 'trainset_question.pickle', 'wb') as handle:
    pickle.dump(train_q, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
file_name = DATA_PATH + "dev-v1.0.json"
with open(file_name) as json_data:
    dev_set = json.load(json_data)
    dev_set = dev_set['data']

In [ ]:
dev_c, dev_q = get_cleaned_dataset(dev_set)

In [ ]:
with open(DATA_PATH + 'devset_context.pickle', 'wb') as handle:
    pickle.dump(dev_c, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(DATA_PATH + 'devset_question.pickle', 'wb') as handle:
    pickle.dump(dev_q, handle, protocol=pickle.HIGHEST_PROTOCOL)